In [1]:
import sounddevice as sd
import numpy as np
import speech_features as sp

In [2]:
import keras
from keras import backend as K
from keras.models import Sequential,Input,Model,load_model
from keras.layers import Conv2D,Conv1D,MaxPooling2D,AveragePooling1D,MaxPooling1D
from keras.layers import Dense,Flatten,Dropout
from keras import initializers,optimizers,backend as k
from keras_radam import RAdam
from keras.callbacks import EarlyStopping,ModelCheckpoint
from sklearn.metrics import confusion_matrix,accuracy_score
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

Using TensorFlow backend.


NameError: name 'os' is not defined

In [24]:
def Classifier(i,hm,optimizer,op):
    x = Input(shape=(40,98,1))
    x1 = (Conv2D(int(hm[0,i]),kernel_size=(40,int(hm[1,i])),activation='relu'))(x)
    x1 = Dropout(rate=hm[8,i])(x1)
    x1 = (MaxPooling2D(pool_size=(1,int(hm[2,i]))))(x1)
    for j in range(int(hm[3,i])):
        x1 = (Conv2D(int(hm[4,i]),kernel_size=(1,int(hm[5,i])),activation='relu'))(x1)
        x1 = Dropout(rate=hm[8,i])(x1)
        x1 = (MaxPooling2D(pool_size=(1,int(hm[6,i]))))(x1)
    y1 = Flatten()(x1)
    y2 = (Dense(int(hm[7,i]),activation='relu'))(y1)
    y2 = Dropout(rate=hm[8,i])(y2)
    y = (Dense(31,activation='softmax'))(y2)
    classifier=Model(inputs=x,outputs=y)
    if optimizer=='radam':
        Optimizer=RAdam()
    elif optimizer=='sgd':
        if op[3]==0:
            Optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
        else:
            Optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=True)
    elif optimizer=='rmsprop':
        Optimizer=keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
    elif optimizer=='adagrad':
        Optimizer='adagrad'
    elif optimizer=='adadelta':
        Optimizer=keras.optimizers.Adadelta(lr=1.0, rho=0.95,epsilon=None, decay=0.0)
    elif optimizer=='adam':
        Optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    elif optimizer=='adamax':
        Optimizer=keras.optimizers.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
    else:
        Optimizer=keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
    classifier.compile(optimizer=Optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
    return classifier

In [42]:
final = np.load('./models/dataset1/CNN1/final.npy')
variables = 9
best_hp = np.zeros((variables,1))
for i in range(variables):
    best_hp[i]=final[i]

optimizer="radam"
op = [[],[0.01,0,0,0],[0.001,0.9,0,0],[0.01,0,0],[1,0.95,0,0],[0.001,0.9,0.999,0,0,0],
      [0.002,0.9,0.999,0,0],[0.002,0.9,0.999,0,0.004]]
    
classifier=Classifier(0,best_hp,optimizer,op[0])
classifier.load_weights('./models/dataset1/CNN1/best_model_radam.h5')

ValueError: Dimension 0 in both shapes must be equal, but are 40 and 44. Shapes are [40,4,1,44] and [44,1,39,4]. for 'Assign_29' (op: 'Assign') with input shapes: [40,4,1,44], [44,1,39,4].

In [37]:
fs = 16000
seconds = 3

myrec = sd.rec(int(fs*seconds), samplerate=fs, channels=1)
sd.wait()

In [41]:
offset = 1000
frame_size = 16000
current_start = 0
i = 0
samples = []

while (current_start + frame_size <= len(myrec)):
    i += 1
    #print(i, current_start, ";", (frame_size + current_start))
    samples.append(myrec[current_start:frame_size + current_start])
    current_start += offset
    
t = []
for s in samples:
    fb = np.array(sp.logfbank(s, fs))
    fb = fb[:-1, :].T
    t.append(fb)
    
t = np.array(t)
t = t.reshape(-1,40,98,1)
print(t.shape)

p = classifier.predict(t)

words_name = ["bed", "bird", "cat", "dog", "down", "fight", "five", "four", "go", "happy",
             "house", "left", "marvin", "nine", "no", "off", "on", "one", "right", "seven",
             "sheila", "silence", "six", "stop", "three", "tree", "two", "up", "wow", "yes",
              "zero"]
for pred in p:
    max_index = np.argmax(pred)
    prob = round((pred[max_index] * 100), 2)
    if prob > 0:
        print("'" + words_name[max_index] + "' with probability " + str(prob) + "%")
    else:
        print("SILENCE")

(33, 40, 98, 1)
'silence' with probability 7.46%
'silence' with probability 7.46%
'silence' with probability 7.46%
'silence' with probability 7.46%
'silence' with probability 7.46%
'silence' with probability 7.46%
'silence' with probability 7.46%
'silence' with probability 7.46%
'silence' with probability 7.46%
'silence' with probability 7.46%
'silence' with probability 7.46%
'silence' with probability 7.46%
'silence' with probability 7.46%
'silence' with probability 6.94%
'silence' with probability 6.53%
'silence' with probability 6.99%
'silence' with probability 7.62%
'silence' with probability 7.03%
'silence' with probability 7.03%
'silence' with probability 6.8%
'silence' with probability 7.16%
'silence' with probability 7.05%
'silence' with probability 8.36%
'silence' with probability 10.48%
'silence' with probability 8.38%
'silence' with probability 6.75%
'silence' with probability 8.21%
'silence' with probability 7.46%
'silence' with probability 6.33%
'silence' with probability 

In [31]:
sd.play(myrec, fs)